In [ ]:
!pip install dask[dataframe] # More efficient than pandas 

In [ ]:
import pandas as pd
import os
import dask.dataframe as dd
import csv
import spacy
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm", exclude=["ner", "tagger", "parser", "lemmatizer", "textcat", "attribute_ruler"]) # loading the tokenizer 
nlp.max_length = 2500000
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/MSMARCO/'
dir = '/content/drive/MyDrive/'

In [ ]:
# read the original collection file
Collection = dd.read_csv(os.path.join(data_dir, 'msmarco-docs.tsv'), sep='\t', header=None, usecols=['docid', 'doc_text'],
                       names=['docid', 'url', 'title', 'doc_text']).compute()

In [ ]:
Passage_Collection = []
    
for i in tqdm(range(len(Collection))):
    new_doc_tokens = []
    count = 0
    tokens = nlp(str(Collection.iloc[i].doc_text)) # tokenize to get a list of tokens 
    for token in tokens:
        if count == 500:
            break
        if token.is_space is False:
            new_doc_tokens.append(token.text) # add tokens until reaching 500
            count = count + 1
    Passage_Collection.append([Collection.iloc[i].docid, ' '.join(new_doc_tokens)])

In [ ]:
# Writing to disk
columns=['docid','doc_text']
with open(os.path.join(data_dir,'Passage_Collection.csv'), 'w', encoding="utf8", newline='') as f:
    write = csv.writer(f)
    write.writerow(columns)
    write.writerows(Passage_Collection)